In [1]:
# Install relevant libraries
!pip install chromadb==0.5.5 \
    langchain-chroma==0.1.2 \
    langchain==0.2.11 \
    langchain-community==0.2.10 \
    langchain-text-splitters==0.2.2 \
    langchain-groq==0.1.6 \
    transformers==4.43.2 \
    sentence-transformers==2.7.0 \
    unstructured==0.15.0 \
    "unstructured[pdf]==0.15.0" \
    gradio \
    pydantic-settings


  Using cached chromadb-0.5.5-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain_chroma-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain-0.2.11-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.2.10-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached langchain_groq-0.1.6-py3-none-any.whl.metadata (2.8 kB)
  Using cached transformers-4.43.2-py3-none-any.whl.metadata (43 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached unstructured-0.15.0-py3-none-any.whl.metadata (29 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached chroma_hnswlib-0.7.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached posthog-6.5.0-py3-none-any.whl.metadata (

In [27]:
#importing relevant libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
import gradio as gr
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import time
import random


In [2]:
#for handling PDF files in the pdf2image library.
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.9 [186 kB]
Fetched 186 kB in 1s (258 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.9_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.9) ...
Setting up poppler-utils (22.02.0-2ubuntu0.9) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
from config import settings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Create a variable for your api key
groq_api_key = settings.groq_api_key

In [12]:
from google.colab import files
import os

# Create a folder for uploaded PDFs
os.makedirs("pdfs", exist_ok=True)

# Upload PDFs
uploaded_files = files.upload()


Saving ai and the future of humans.pdf to ai and the future of humans.pdf
Saving Artificial Intelligence Accelerates Human Learning_ Discussion Data Analytics ( PDFDrive ).pdf to Artificial Intelligence Accelerates Human Learning_ Discussion Data Analytics ( PDFDrive ).pdf
Saving artificial intelligence.pdf to artificial intelligence.pdf
Saving future of A.I intelligence.pdf to future of A.I intelligence.pdf
Saving Introduction to Artificial Intelligence ( PDFDrive ).pdf to Introduction to Artificial Intelligence ( PDFDrive ).pdf
Saving preparing for future of ai.pdf to preparing for future of ai.pdf
Saving Stahl_2021_Artificial_intelligence_for_human_f.pdf to Stahl_2021_Artificial_intelligence_for_human_f.pdf
Saving the future of AI.pdf to the future of AI.pdf


In [13]:
# Save them into "pdfs" folder
for filename in uploaded_files.keys():
    os.rename(filename, os.path.join("pdfs", filename))

print(f"Uploaded {len(uploaded_files)} PDF(s) successfully.")


Uploaded 8 PDF(s) successfully.


In [14]:
#Load PDFs into LangChain
all_docs = []

# Loop through PDFs in the folder
for pdf_file in os.listdir("pdfs"):
    if pdf_file.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join("pdfs", pdf_file))
        docs = loader.load()
        all_docs.extend(docs)

print(f"Loaded {len(all_docs)} pages from all PDFs.")


Loaded 821 pages from all PDFs.


In [15]:
all_docs

[Document(metadata={'source': 'pdfs/the future of AI.pdf', 'page': 0}, page_content='The Future of Artificial Intelligence\nCHAPTER FOURTEEN\nDavid A. Bray\n“AI can match humans into different ad-hoc teams…to fit a specific \npublic service goal or problem set. If an emergent event or crisis \noccurs, AI can help identify who is available to assist with what \nactivities—and even help coordinate swarming activities of both \nhumans and machines to assist with the response to the event. AI \ncan learn which humans work better on specific tasks with other \nhumans…”'),
 Document(metadata={'source': 'pdfs/the future of AI.pdf', 'page': 1}, page_content='THE FUTURE OF ARTIFICIAL INTELLIGENCE\nBy David A. Bray\nImagine being able to visit a disability claims office in a digital environment. \nImagine a patent examiner equipped with digital assistants that could do the \nbulk of administrative work behind processing patents. Artificial intelligence \n(AI) may make both of these scenarios a r

In [16]:
#Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(all_docs)

print(f"Split into {len(chunks)} chunks.")


Split into 2736 chunks.


In [17]:
#Get the first 5 text chunks
print(chunks[0])
print("-----------------------------------------------------------------------")
print("-----------------------------------------------------------------------")
print(chunks[5])

page_content='The Future of Artificial Intelligence
CHAPTER FOURTEEN
David A. Bray
“AI can match humans into different ad-hoc teams…to fit a specific 
public service goal or problem set. If an emergent event or crisis 
occurs, AI can help identify who is available to assist with what 
activities—and even help coordinate swarming activities of both 
humans and machines to assist with the response to the event. AI 
can learn which humans work better on specific tasks with other 
humans…”' metadata={'source': 'pdfs/the future of AI.pdf', 'page': 0}
-----------------------------------------------------------------------
-----------------------------------------------------------------------
page_content='question being asked. Such AI assistance will also help government employ-
ees with questions about onboarding, starting a new role, help with an exist-
ing role, retiring, and other internal service queries. 
Increased Use of AI-Supported Assistance for Talent Management and 
Skills Match

In [19]:
# Instantiate embedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#instantiate vectorstore
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_db"
)



ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [20]:
# ===== SETTINGS =====
persist_directory = "chroma_db"
first_run = False  # 🔹 Change to True only when adding new PDFs

# ===== EMBEDDINGS =====
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if first_run:
    # 1️⃣ Create & Save Vectors
    # Make sure `chunks` is already defined from your document loader + text splitter
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vectorstore.persist()
    print("✅ Vector DB created and saved.")
else:
    # 2️⃣ Load Existing Vectors
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )
    print("✅ Vector DB loaded from disk.")

# ===== RETRIEVER =====
retriever = vectorstore.as_retriever()


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ Vector DB loaded from disk.


In [ ]:
# set-up retriever to retrieve information from our vector database
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [21]:
# Create a llm from groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.5,
    groq_api_key=groq_api_key
)

In [23]:
# Create a conversational chain
conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [24]:
# Invoke the conversational chain to ask our question and get a response
question = "What is the impact of ai?"
response = conv_chain.invoke({"question": question, "chat_history": []})
print(f"Answer: {response['answer']}")
print(f"Source Document: {response['source_documents']}")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Answer: According to the context provided, the impact of AI can be both positive and negative. Some experts believe that AI will:

* Improve human health and disposition
* Enhance human-human interaction across space and time
* Aid, augment, and amplify individual and collective human intelligence
* Reduce human error in many areas
* Help mankind be more efficient, live safer and healthier, and manage resources more efficiently

However, others also warn that AI may:

* Reduce autonomy and increase social control
* Be used by those in power to manipulate, control, and dominate others
* Be biased towards those with more resources
* Reduce privacy
* Create multiple realities, making it hard to discern the truth
* Be used for profit or power at the expense of individuals and the betterment of the human condition

Overall, the impact of AI will depend on how it is governed and used, and whether its development and implementation prioritize human well-being and values.
Source Document: [Doc

In [25]:

# Create function to process the user question with other necessary information
def process_question(user_question, history):
    try:
        start_time = time.time()

        if history is None:
            history = []

        # Prepare chat_history in the format expected by conv_chain
        chat_history = [(h[0], h[1].split("\n\nResponse time:")[0]) for h in history]

        # Debug print
        print(f"Processing question: {user_question}")
        print(f"Chat history: {chat_history}")

        # Custom response for ownership/founder-related questions
        if "founder" in user_question.lower() or "builder" in user_question.lower() or "owner" in user_question.lower():
            response = (
                "This chat assistant was built and is maintained by Mr. Joel Tamakloe, a data scientist and an AI and Cybersecurity enthusiast. "
                "Joel's background includes extensive experience in building AI-powered applications and solving real-world problems using data. "
                "This assistant was created to make information about artificial intelligence more accessible and to assist users in exploring AI concepts interactively. "
                "It is powered by advanced AI models like LLaMA and uses cutting-edge tools such as Hugging Face for embedding and Chroma for vector storage. "
                "Currently, it's in the testing phase with a focus on AI-related topics, aiming to improve its capabilities and expand into educational and business applications in the future."
            )

        # Custom response for personal questions like "Tell me about yourself"
        elif "who are you" in user_question.lower() or "myself" in user_question.lower() or "yourself" in user_question.lower():
            response = (
                "I am an AI-powered chat assistant designed to assist users with exploring and learning about artificial intelligence and related topics. "
                "My purpose is to provide an intuitive way for users to interact with AI and gain insights on topics related to artificial intelligence. "
                "I use advanced tools and technologies like the LLaMA model, a powerful large language model, to process natural language queries, and Chroma, a vector database management system, to efficiently store and retrieve information. "
                "In the future, I plan to expand my abilities to cover more topics, improve response accuracy, and possibly integrate video and voice interaction for a more dynamic user experience. "
                "Ask me anything about A.I. I am happy to help! 😊"
            )

        # Custom response for greetings like "Hello"
        elif "hello" in user_question.lower() or "hi" in user_question.lower():
            response = "Hello! How can I assist you today?"

        # Invoke conv_chain with both the question and chat_history for all other questions
        else:
            response = conv_chain.invoke({"question": user_question, "chat_history": chat_history})

            # If response is a dict, extract the actual response text
            if isinstance(response, dict) and 'answer' in response:
                response = response['answer']

            # Add a conversational prompt occasionally
            if random.random() < 0.3:  # 30% chance to add small talk
                response += "\n\nBy the way, feel free to ask me anything else or even chat casually!"

        # Measure the response time
        end_time = time.time()
        response_time = f"Response time: {end_time - start_time:.2f} seconds."

        # Combine the response and the response time
        full_response = f"{response}\n\n{response_time}"

        # Update the history
        history.append((user_question, full_response))

        # Debug print
        print(f"Processed successfully. Response: {full_response}")

        return history,history, full_response

    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        return history, history, error_message


In [26]:
# Setup the Gradio interface
iface = gr.Interface(
    fn=process_question,
    inputs=[
        gr.Textbox(lines=2, placeholder="Type your question here..."),
        gr.State()
    ],
    outputs=[
        gr.Chatbot(),
        gr.State(),
        gr.Textbox(label="Latest Answer")
    ],
    title="Joel's A.I Chat Assistant",
    description="Ask any question about Artificial Intelligence."
)

# Launch the interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://83cc587ba02edfe65a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
